# Example of events generation
## 1. Generate python data structure

In [1]:
from helixgen.phspdecay import generate
import jax
import jax.numpy as np
import jax.random as rjax
rng = rjax.PRNGKey(seed=0)

/home/vitaly/miniconda3/envs/tf2/lib/python3.8/site-packages/jax/lib/xla_bridge.py:125: UserWarning: No GPU/TPU found, falling back to CPU.
  warnings.warn('No GPU/TPU found, falling back to CPU.')


In [2]:
w, events = generate(rng, '', 10)

In [3]:
jax.tree_map(np.shape, events)

{'D0_pi+': {'hel': Helix(d0=(10,), phi0=(10,), omega=(10,), z0=(10,), tanl=(10,)),
  'helcov': (10, 5, 5),
  'meas_mom': Momentum(px=(10,), py=(10,), pz=(10,)),
  'meas_pos': Position(x=(10,), y=(10,), z=(10,)),
  'mom': Momentum(px=(10,), py=(10,), pz=(10,)),
  'pcl': (),
  'pos': Position(x=(10,), y=(10,), z=(10,))},
 'K*0': {'mom': Momentum(px=(10,), py=(10,), pz=(10,)),
  'pcl': (),
  'pos': Position(x=(10,), y=(10,), z=(10,))},
 'K-': {'hel': Helix(d0=(10,), phi0=(10,), omega=(10,), z0=(10,), tanl=(10,)),
  'helcov': (10, 5, 5),
  'meas_mom': Momentum(px=(10,), py=(10,), pz=(10,)),
  'meas_pos': Position(x=(10,), y=(10,), z=(10,)),
  'mom': Momentum(px=(10,), py=(10,), pz=(10,)),
  'pcl': (),
  'pos': Position(x=(10,), y=(10,), z=(10,))},
 'pi+': {'hel': Helix(d0=(10,), phi0=(10,), omega=(10,), z0=(10,), tanl=(10,)),
  'helcov': (10, 5, 5),
  'meas_mom': Momentum(px=(10,), py=(10,), pz=(10,)),
  'meas_pos': Position(x=(10,), y=(10,), z=(10,)),
  'mom': Momentum(px=(10,), py=(10,),

$K^-$ helix can be obtained like this

In [4]:
events['K-']['hel'].as_array

DeviceArray([[ 1.8784378e-03,  2.4707544e+00, -7.2642660e-01,
               4.6566039e-02, -1.2927436e+00],
             [ 1.8784378e-03, -5.8064216e-01, -4.3403559e+00,
               2.7084833e-01, -1.1174452e+01],
             [ 1.8784035e-03, -5.1766461e-01, -2.2919002e+00,
               1.4317392e-01, -6.9602547e+00],
             [ 1.8784378e-03, -2.4175730e+00, -4.5019540e-01,
               3.0302199e-02,  4.8053306e-01],
             [ 1.8784378e-03, -2.0075829e+00, -5.6167579e-01,
               3.6757685e-02,  8.5757804e-01],
             [ 1.8784378e-03,  1.1063408e+00, -3.9014509e-01,
               2.6929714e-02, -4.3492144e-01],
             [ 1.8784378e-03,  1.3395319e+00, -4.2565790e-01,
               2.8912904e-02, -7.8298289e-01],
             [ 1.8784378e-03,  2.6951516e-01, -1.4496518e+00,
               9.0889499e-02,  1.7629911e+00],
             [ 1.8784378e-03, -3.3019242e+00, -7.1021038e-01,
               4.5591213e-02,  1.0505638e+00],
             [ 1.87

Let's reproduce `generate` function step by step

## 2. Generate json file

In [5]:
from helixgen.mcproducer import produce_and_serialize_to_json
import json

In [6]:
produce_and_serialize_to_json(rng, decstr='', nevts=10, lbl='dkstpi')

In [7]:
with open('dkstpi.json', 'r') as f:
    evts = json.loads(f.read())

Get the same $K^-$ helix

In [8]:
np.array(evts['K-']['hel'])

DeviceArray([[ 1.8784378e-03,  2.6390305e+00, -4.6821249e-01,
               3.1330813e-02,  1.0649364e+00],
             [ 1.8784378e-03,  2.8133566e+00, -1.9676927e+00,
               1.2301609e-01,  1.3805670e+00],
             [ 1.8784378e-03,  3.4768680e-01, -3.4942749e-01,
               2.4705028e-02,  1.1730474e-01],
             [ 1.8784378e-03,  2.0340087e+00, -1.4680680e+00,
               9.2028856e-02, -3.0428123e-01],
             [ 1.8784378e-03, -1.3022797e-01, -6.4299393e-01,
               4.1569877e-02, -9.0085171e-02],
             [ 1.8785064e-03, -1.6025252e+00, -1.2171190e+00,
               7.6530941e-02, -3.3130622e+00],
             [ 1.8784378e-03, -1.5317198e+00, -1.0310783e+00,
               6.5095574e-02, -1.6932867e+00],
             [ 1.8784378e-03,  2.9218659e+00, -4.5383522e-01,
               3.0509450e-02, -1.0607923e+00],
             [ 1.8784378e-03, -1.1591259e+00, -3.9068028e-01,
               2.6959332e-02,  9.5540810e-01],
             [ 1.87